In [ ]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import base64
import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter
from animal_shelter import AnimalShelter




###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "accuser"
password = "Lily"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.readAll({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Cinnamon George'))),
    html.Hr(),
    html.Div(
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.

        editable=True,
        filter_action="native", # this refers to data table built in and needs to be one, such as native
        sort_action="native",  # same issue here
        sort_mode="multi",
        column_selectable=False,  # can only be True or False
        row_selectable="single",   # not an error, but single will work better for the project
        row_deletable=False, # not an error, but False will work better for the project
        selected_columns=[],
        selected_rows=[0], # needs to select the top row for the map to display on first load of application
        page_actions="native", # this refers to data table built in and needs to be one, such as native
         page_current= 0,
         page_size= 10,
    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries

        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
def update_styles(selected_columns):
    return [{
#          'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(Output('datatable-interactivity', "data"),
             [Input('submit-button-one', 'n_clicks'),Input('submit-button-two', 'n_clicks'),
              ])
def on_click(bt1, bt2):
# # start case
    if (int(bt1) == 0 and int(bt2) == 0):
        df = pd.DataFrame.from_records(shelter.readAll({}))
     # use higher number of button clicks to determine filter type
    elif (intbt1) > int(bt2):
         df = pd.DataFrame(list(shelter.readAll({"animal_type": "Cat"})))
    elif (int(bt2) > int(bt1)):
         df = pd.DataFrame(list(shelter.readAll({"animal_type": "Dog"})))
    
                
    return df.to_dict("records");
    return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) # example
    #return [
    #    dcc.Graph(            
    #        figure = ###
    #    )    
    #]
    import chart_studio.plotly as py
    import plotly.graph_objects as go

    import pandas as pd

# mapbox_access_token = 'ADD YOUR TOKEN HERE'

df = pd.read_csv("AAC - Georgraphical Location")
site_lat = df.lat
site_lon = df.lon
locations_name = df.text

data = [
    go.Scattermapbox(
        lat=site_lat,
        lon=site_lon,
        mode='markers',
        marker=dict(
            size=17,
            color='rgb(255, 0, 0)',
            opacity=0.7
        ),
        text=locations_name,
        hoverinfo='text'
    ),
    go.Scattermapbox(
        lat=site_lat,
        lon=site_lon,
        mode='markers',
        marker=dict(
            size=8,
            color='rgb(242, 177, 172)',
            opacity=0.7
        ),
        hoverinfo='none'
    )]


layout = go.Layout(
    title='Animal Locater',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=30.75,
            lon=-97.48
        ),
        pitch=0,
        zoom=3,
        style='light'
    ),
)

fig = dict(data=data, layout=layout)

py.iplot(fig, filename='AAC')

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
     # Austin TX is at [30.75,-97.48]
        
    return [
         dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
             dl.TileLayer(id="base-layer-id"),
                 # Marker with tool tip and popup
                 dl.Marker(position=[30.75,-97.48], children=[
                     dl.Tooltip(dff.iloc[0,4]),
                         dl.Popup([
                             html.H1("Animal Name"),
                             html.P(dff.iloc[1,9])
                 ])
             ])
         ])
     ]


app